In [3]:
path = "C:\\Users\\tajdi\\Downloads\\project Idriss\\project Idriss\\data.xlsx"

In [6]:
from excel_manager import create_dep_datetime, filter_tec
import polars as pl

In [8]:
df = pl.read_excel(path, sheet_name="Sheet1").lazy()

In [15]:
df = df.pipe(filter_tec).pipe(create_dep_datetime)

TypeError: datetime_() missing 1 required positional argument: 'day'

In [ ]:
def create_dep_datetime(df_lazy: pl.LazyFrame) -> pl.LazyFrame:
    return df_lazy.with_columns(
        pl.col("DEP_DAY_SCHED")
        .dt.combine(pl.col("DEP_TIME_SCHED").str.strptime(pl.Time, "%H:%M"))
        .alias("DEP_DATETIME")
    )